In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('test_data_clean_uak211.csv')

In [3]:
data.head()

,Unnamed: 0,date,station_id,test_type,test_result,vin,model_year,vehicle_type,vehicle_class,gross_vehicle_weight_rating
0,1,01/01/2017,456.0,OBDII,Pass,5N1AN08W97C530793,2007.0,Truck,LDT1,5400.0
1,2,01/01/2017,1115.0,OBDII,Pass,WBANB53537CP05224,2007.0,Passenger Car,LDV,5026.0
2,3,01/01/2017,1109.0,OBDII,Pass,JN8AZ28R49T111532,2009.0,Passenger Car,LDT1,3848.0
3,4,01/01/2017,1115.0,OBDII,Pass,1N6AA07C18N350248,2008.0,Truck,LDT1,7200.0
4,5,01/01/2017,1115.0,Idle,Fail,1J4GZ88S5PC113745,1993.0,Passenger Car,LDV,5500.0


## 1. Prepare OBDII test only data
    - filter out the OBDII data
    - merge the OBDII data with verified csv (with price for further use)
    - merge the OBDII data with station location (lat/long) for mapping

In [4]:
OBDII_data = data[data['test_type'] == 'OBDII']

In [5]:
OBDII_data = pd.DataFrame(OBDII_data.groupby(['station_id'])['test_result'].count()).reset_index()

In [6]:
OBDII_data.head()

,station_id,test_result
0,1.0,2099
1,2.0,1916
2,11.0,627
3,24.0,1515
4,28.0,1029


In [7]:
station_info = pd.read_csv('verified_avg_fee_by_station_jx.csv')
station_info.head()

,No.,Station Id,Avg. Test Fees,Station Name,Address,City,ZIP,Phone Number,Price,Tax,On the Spot Registration,Registration/Sticker Fee,Service Type,Detailed Information
0,1.0,2.0,$25,KEN GARFF MERCEDES BENZ,575 SOUTH STATE,SALT LAKE CITY,84111.0,855-780-1053,25,0,Na,Na,TestAndRepair,Na
1,2.0,24.0,$39,OLYMPUS HILLS SINCLAIR,3905 SOUTH WASATCH BLVD,SALT LAKE CITY,84124.0,801- 272-2081,39,0,Na,Na,TestAndRepair,Na
2,3.0,28.0,$31,UNION SERVICE,997 EAST 7220 SOUTH,MIDVALE,84047.0,801-255-4122,Na,Na,Na,Na,Na,Na
3,4.0,38.0,$30,LARRY H. MILLER TOYOTA,5650 S STATE ST,MURRAY,84107.0,801-264-3850,30,0,Yes,5,TestAndRepair,Na
4,5.0,46.0,$35,QUALITY TIRE CO.,1335 WEST 2100 SOUTH,SALT LAKE CITY,84119.0,801-972-1944,Na,Na,Na,Na,Na,Na


In [8]:
OBDII_data = pd.merge(OBDII_data, station_info, how='left', left_on ='station_id', right_on = 'Station Id')

In [9]:
OBDII_data.shape

(522, 16)

In [10]:
station_latlng = pd.read_csv('SLCo_station_latlong_v1_yc.csv')
station_latlng = station_latlng[['Station Id', 'Latitude', 'Longitude']]
station_latlng['Station Id'] = station_latlng['Station Id'].astype(int)
station_latlng.head()

,Station Id,Latitude,Longitude
0,2,40.756698,-111.887756
1,24,40.686708,-111.795525
2,28,40.619111,-111.862117
3,38,40.648415,-111.889550
4,46,40.725605,-111.929081


In [11]:
OBDII_data = pd.merge(OBDII_data, station_latlng, left_on = 'station_id', right_on = 'Station Id', how='right')
OBDII_data.shape

(435, 19)

In [12]:
OBDII_data.to_csv('mapping_OBDII_data_yc3420.csv')

## 2. Prepare all test type
    - merge all test data with verified csv (with price for further use)
    - merge all test data with station location (lat/long) for mapping

In [13]:
All_data = pd.DataFrame(data.groupby(['station_id'])['test_result'].count()).reset_index()

In [14]:
All_data.head()

,station_id,test_result
0,1.0,2938
1,2.0,1961
2,11.0,797
3,24.0,1635
4,28.0,1167


In [15]:
All_data = pd.merge(All_data, station_info, how='left', left_on ='station_id', right_on = 'Station Id')

In [16]:
All_data = pd.merge(All_data, station_latlng, left_on = 'station_id', right_on = 'Station Id', how='right')

In [17]:
All_data.head()

,station_id,test_result,No.,Station Id_x,Avg. Test Fees,Station Name,Address,City,ZIP,Phone Number,Price,Tax,On the Spot Registration,Registration/Sticker Fee,Service Type,Detailed Information,Station Id_y,Latitude,Longitude
0,2.0,1961.0,1.0,2.0,$25,KEN GARFF MERCEDES BENZ,575 SOUTH STATE,SALT LAKE CITY,84111.0,855-780-1053,25,0,Na,Na,TestAndRepair,Na,2,40.756698,-111.887756
1,24.0,1635.0,2.0,24.0,$39,OLYMPUS HILLS SINCLAIR,3905 SOUTH WASATCH BLVD,SALT LAKE CITY,84124.0,801- 272-2081,39,0,Na,Na,TestAndRepair,Na,24,40.686708,-111.795525
2,28.0,1167.0,3.0,28.0,$31,UNION SERVICE,997 EAST 7220 SOUTH,MIDVALE,84047.0,801-255-4122,Na,Na,Na,Na,Na,Na,28,40.619111,-111.862117
3,38.0,11557.0,4.0,38.0,$30,LARRY H. MILLER TOYOTA,5650 S STATE ST,MURRAY,84107.0,801-264-3850,30,0,Yes,5,TestAndRepair,Na,38,40.648415,-111.889550
4,46.0,4954.0,5.0,46.0,$35,QUALITY TIRE CO.,1335 WEST 2100 SOUTH,SALT LAKE CITY,84119.0,801-972-1944,Na,Na,Na,Na,Na,Na,46,40.725605,-111.929081


In [18]:
All_data.to_csv('mapping_All_data_yc3420.csv')

# 3. Download and clean the population data

### data source: American Fact Finder
### 2013-2017 American Community Survey 5-Year Estimates
### instruction link: https://gis.stackexchange.com/questions/9380/where-to-get-2010-census-block-data

In [21]:
SLCo_pop_all = pd.read_csv('aff_download/ACS_17_5YR_DP05_with_ann.csv')

In [22]:
SLCo_pop_all.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_VC03,HC02_VC03,HC03_VC03,HC04_VC03,HC01_VC04,HC02_VC04,HC03_VC04,...,HC03_VC113,HC04_VC113,HC01_VC114,HC02_VC114,HC03_VC114,HC04_VC114,HC01_VC115,HC02_VC115,HC03_VC115,HC04_VC115
0,Id,Id2,Geography,Estimate; SEX AND AGE - Total population,Margin of Error; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Percent Margin of Error; SEX AND AGE - Total p...,Estimate; SEX AND AGE - Total population - Male,Margin of Error; SEX AND AGE - Total populatio...,Percent; SEX AND AGE - Total population - Male,...,"Percent; CITIZEN, VOTING AGE POPULATION - Citi...","Percent Margin of Error; CITIZEN, VOTING AGE P...","Estimate; CITIZEN, VOTING AGE POPULATION - Cit...","Margin of Error; CITIZEN, VOTING AGE POPULATIO...","Percent; CITIZEN, VOTING AGE POPULATION - Citi...","Percent Margin of Error; CITIZEN, VOTING AGE P...","Estimate; CITIZEN, VOTING AGE POPULATION - Cit...","Margin of Error; CITIZEN, VOTING AGE POPULATIO...","Percent; CITIZEN, VOTING AGE POPULATION - Citi...","Percent Margin of Error; CITIZEN, VOTING AGE P..."
1,1400000US49035100100,49035100100,"Census Tract 1001, Salt Lake County, Utah",1690,249,1690,(X),1033,202,61.1,...,1210,(X),723,166,59.8,7.9,487,125,40.2,7.9
2,1400000US49035100200,49035100200,"Census Tract 1002, Salt Lake County, Utah",1405,142,1405,(X),672,93,47.8,...,1132,(X),573,87,50.6,4.6,559,91,49.4,4.6
3,1400000US49035100306,49035100306,"Census Tract 1003.06, Salt Lake County, Utah",5219,512,5219,(X),2519,334,48.3,...,2628,(X),1434,272,54.6,6.9,1194,230,45.4,6.9
4,1400000US49035100307,49035100307,"Census Tract 1003.07, Salt Lake County, Utah",5487,457,5487,(X),2567,298,46.8,...,2485,(X),1259,165,50.7,4.3,1226,176,49.3,4.3


In [43]:
SLCo_pop = SLCo_pop_all.iloc[1:,1:4]

In [44]:
SLCo_pop.head()

,GEO.id2,GEO.display-label,HC01_VC03
1,49035100100,"Census Tract 1001, Salt Lake County, Utah",1690
2,49035100200,"Census Tract 1002, Salt Lake County, Utah",1405
3,49035100306,"Census Tract 1003.06, Salt Lake County, Utah",5219
4,49035100307,"Census Tract 1003.07, Salt Lake County, Utah",5487
5,49035100308,"Census Tract 1003.08, Salt Lake County, Utah",4274


In [45]:
SLCo_pop = SLCo_pop.set_axis(['GEOID','Geo_Label','tot_pop'], inplace=False, axis=1)

In [47]:
SLCo_pop.head()

,GEOID,Geo_Label,tot_pop
1,49035100100,"Census Tract 1001, Salt Lake County, Utah",1690
2,49035100200,"Census Tract 1002, Salt Lake County, Utah",1405
3,49035100306,"Census Tract 1003.06, Salt Lake County, Utah",5219
4,49035100307,"Census Tract 1003.07, Salt Lake County, Utah",5487
5,49035100308,"Census Tract 1003.08, Salt Lake County, Utah",4274


In [48]:
SLCo_pop.to_csv('SLCo_pop.csv')